In [1]:
DATASET_DIR = './dataset/'

## Prelude

In [2]:
from tqdm import tqdm
import open3d as o3d
import numpy as np
import PIL
import cv2
import pathlib
import matplotlib.pyplot as plt

## Utilities

In [3]:
from functools import reduce

flat_map = lambda f, xs: reduce(lambda a, b: a + b, map(f, xs))

In [4]:
get_dirs = lambda p: [d for d in p.iterdir() if d.is_dir()]

def map_dataset(f: callable, dataset_dir: str):
    ds = pathlib.Path(dataset_dir)
    
    gestures = get_dirs(ds)
    fingers = flat_map(get_dirs, gestures)
    subjects = flat_map(get_dirs, fingers)
    essais = flat_map(get_dirs, subjects)
    
    for es in tqdm(essais): f(es)

In [5]:
png_iter = lambda p: [f for f in p.iterdir() if f.suffix == '.png']
png_sorter = lambda pngs: sorted(pngs, key=lambda f: int(f.name.split('_')[0]))

## PCD Generation

In [6]:
def to_pcd(fn: str):
    depth_raw = o3d.io.read_image(fn)
    pcd = o3d.geometry.PointCloud.create_from_depth_image(
        depth_raw,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
        )
    )
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    return pcd

In [7]:
def pcd_map_essai(essai_dir: pathlib.Path):
    pngs = png_iter(essai_dir)
    for f in pngs:
        pcd = to_pcd(str(f))
        ofname = str( f.parent / (f.name + '.pcd') )
        o3d.io.write_point_cloud(ofname, pcd, print_progress=True)
        del pcd

In [8]:
run_pcd_mapper = lambda ds: map_dataset(pcd_map_essai, ds)

## GIF Creation

In [10]:
def dnorm(x):
    x = (x - x.min()) / (x.max() - x.min())
    x = x * 255.
    return x

In [9]:
def gif_map_essai(essai_dir: pathlib.Path):
    pngs = png_iter(essai_dir)
    pngs = png_sorter(pngs)
    nls = list()
    for f in pngs:
        x = cv2.imread(str(f), cv2.IMREAD_ANYDEPTH)
        x = dnorm(x)
        x = np.stack((x,)*3, axis=-1)
        x = x.astype('uint8')
        x = PIL.Image.fromarray(x)
        nls.append(x)
    
    name = str(essai_dir / 'action.gif')
    duration = 1000/30
    nls[0].save(
        name,
        append_images=nls[1:],
        save_all=True,
        duration=duration,
        loop=0
    )

In [11]:
run_gif_mapper = lambda ds: map_dataset(gif_map_essai, ds)

## Run preprocessing

In [12]:
run_pcd_mapper(DATASET_DIR)

100%|██████████| 9/9 [00:10<00:00,  1.22s/it]


In [13]:
run_gif_mapper(DATASET_DIR)

100%|██████████| 9/9 [00:14<00:00,  1.60s/it]
